In [1]:
import pickle

In [2]:
with open('sexual_assault.pkl', 'rb') as f:
    model = pickle.load(f)

In [3]:
model

BinaryRelevance(classifier=SVC(), require_dense=[True, True])

In [2]:
import nltk
nltk.download("stopwords")
# model.predict('[go die]')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pree_samurai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
!pip3 install scikit_multilearn xgboost

zsh:1: /opt/homebrew/bin/pip3: bad interpreter: /opt/homebrew/opt/python@3.11/bin/python3.11: no such file or directory


In [4]:
from nltk.corpus import stopwords

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost
from sklearn.metrics import roc_auc_score, accuracy_score

In [6]:
attack = pd.read_csv("attack_parsed_dataset.csv")
aggression = pd.read_csv("aggression_parsed_dataset.csv")
kaggle = pd.read_csv("kaggle_parsed_dataset.csv")
toxic = pd.read_csv("toxicity_parsed_dataset.csv")
#twitter_racist = pd.read_csv("twitter_racism_parsed_dataset.csv")
#twitter_sexist = pd.read_csv("twitter_sexism_parsed_dataset.csv")
twitter_p = pd.read_csv("twitter_parsed_dataset.csv")
youtube = pd.read_csv("youtube_parsed_dataset.csv")
reddit = pd.read_csv("reddit.csv")

In [7]:
### Only Run when files are newly read
reddit["Text"] = reddit["tweet"]
reddit = reddit.drop(["tweet"], axis=1)
reddit["class"].replace(to_replace = 0, value = 1, inplace=True)
reddit["class"].replace(to_replace = 2, value = 0, inplace=True)
reddit["oh_label"] = reddit["class"].astype(float)
reddit = reddit.drop(["class"], axis=1)
reddit

,Unnamed: 0,count,hate_speech,offensive_language,neither,Text,oh_label
0,0,3,0,0,3,!!! RT @mayasolovely: As a woman you shouldn't...,0.0
1,1,3,0,3,0,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1.0
2,2,3,0,3,0,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1.0
3,3,3,0,2,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1.0
4,4,6,0,6,0,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1.0
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1.0
24779,25292,3,0,1,2,"you've gone and broke the wrong heart baby, an...",0.0
24780,25294,3,0,3,0,young buck wanna eat!!.. dat nigguh like I ain...,1.0
24781,25295,6,0,6,0,youu got wild bitches tellin you lies,1.0


In [9]:
attack = attack[["Text", "oh_label"]]
aggression = aggression[["Text", "oh_label"]]
kaggle = kaggle[["Text", "oh_label"]]
toxic = toxic[["Text", "oh_label"]]
#twitter_racist = twitter_racist[["Text", "oh_label"]]
#twitter_sexist = twitter_sexist[["Text", "oh_label"]]
twitter_p = twitter_p[["Text", "oh_label"]]
youtube = youtube[["Text", "oh_label"]]
reddit = reddit[["Text", "oh_label"]]

In [10]:
#mega_set = pd.concat([attack, aggression, kaggle, toxic, twitter_p, twitter_racist, twitter_sexist, youtube, reddit])
mega_set = pd.concat([attack, aggression, kaggle, toxic, twitter_p, youtube, reddit])
mega_set

,Text,oh_label
0,`- This is not ``creative``. Those are the di...,0.0
1,` :: the term ``standard model`` is itself le...,0.0
2,"True or false, the situation as of March 200...",0.0
3,"Next, maybe you could work on being less cond...",0.0
4,This page will need disambiguation.,0.0
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1.0
24779,"you've gone and broke the wrong heart baby, an...",0.0
24780,young buck wanna eat!!.. dat nigguh like I ain...,1.0
24781,youu got wild bitches tellin you lies,1.0


In [11]:
mega_set.drop_duplicates(subset='Text', keep=False, inplace=True)
mega_set

,Text,oh_label
0,"""You fuck your dad.""",1.0
1,"""i really don't understand your point.\xa0 It ...",0.0
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0.0
3,"""listen if you dont wanna get married to a man...",0.0
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0.0
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1.0
24779,"you've gone and broke the wrong heart baby, an...",0.0
24780,young buck wanna eat!!.. dat nigguh like I ain...,1.0
24781,youu got wild bitches tellin you lies,1.0


In [12]:
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(data):
    stop = stopwords.words('english')
    res = []
    data['Text'] = data['Text'].apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop)]))
    for x in data['Text']:
        print(x)
        x = str(x)
        for punct in puncts:
            if punct in x:
                    x = x.replace(punct,' ')
        res.append(x)
    return res

In [13]:
mega_set["Text"] = clean_text(mega_set)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Yall bitches muslim yall hair fucked ctfu stop youngs!!
Yall bitches girlfriends Halloween since yall sidechiccs year &#128514;&#128557;&#128564;
Yall black ass hoes aint east African
Yall stop RTing whataburger shit cuz I'm still gon eat there. Shouldn't ordered chicken dumb bitch.
Yall wAit till Halloween dress like hoes
Yall want nigga unless bitches want him.. But get get mad cuz bitches still want him. &#128533;?
Yall hoes got nothing better gossip play telephone.
Yall hoes still play around wrong shit I see.. Smh.
Yall know Skip Bayless slit wrists Romo! Lol
Yall letting hoes fuck strap said no?
Yall live bitches blunts. We live weed money!!!!!!
Yall fuckin women's men risking ya life &amp; freedom I'm single &amp; dolo bitch gotta worry about.
Yall see Chojis kid? Big ass black bitch
Yall trustin hoes... I've never gullible
Yandy gotta pay dick im sorry...that bitch ugly
Yankee homeruns make tingly.
Yankee pinstripes might best unis MLB gray road ones worst CC looks like elephan

In [14]:
X = mega_set["Text"]
y = mega_set["oh_label"]

In [15]:
def preprocessing(data):
    res = []
    for each in data['Text']:
        res.append(each.lower())
    return res

In [16]:
mega_set["cleaned"] = preprocessing(mega_set)

In [17]:
mega_set = mega_set.drop(["Text"], axis = 1)

In [18]:
mega_set["cleaned"] = mega_set.apply(lambda r: r['cleaned'] if type(r['cleaned'])==str else "HI", axis=1)
mega_set.dropna(inplace=True)
mega_set

,oh_label,cleaned
0,1.0,you fuck dad
1,0.0,i really understand point xa0 it seems mixin...
2,0.0,a xc2 xa0majority canadians wrong again n...
3,0.0,listen dont wanna get married man women dont ...
4,0.0,c xe1c b u1ea1n xu u1ed1ng u0111 u01b0 u1edd...
...,...,...
24778,1.0,you s muthaf in lie 8220 lifeasking 20 ...
24779,0.0,gone broke wrong heart baby drove redneck crazy
24780,1.0,young buck wanna eat dat nigguh like i ain...
24781,1.0,youu got wild bitches tellin lies


In [19]:
mega_set["cleaned"] = mega_set["cleaned"].astype(str)
mega_set["cleaned"]

0                                           you fuck dad  
1         i really understand point  xa0 it seems mixin...
2         a  xc2  xa0majority canadians wrong again   n...
3         listen dont wanna get married man women dont ...
4         c xe1c b u1ea1n xu u1ed1ng  u0111 u01b0 u1edd...
                               ...                        
24778    you s muthaf   in lie   8220  lifeasking   20 ...
24779     gone broke wrong heart baby  drove redneck crazy
24780    young buck wanna eat     dat nigguh like i ain...
24781                    youu got wild bitches tellin lies
24782      ruffled   ntac eileen dahlia   beautiful col...
Name: cleaned, Length: 134361, dtype: object

In [20]:
X = mega_set["cleaned"]
y = mega_set["oh_label"]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [21]:
y_test = pd.DataFrame(y_test)
y_train = pd.DataFrame(y_train)
y_train, y_test

(        oh_label
 6386         1.0
 9485         1.0
 6726         1.0
 148318       0.0
 117229       0.0
 ...          ...
 79707        0.0
 12924        0.0
 127798       0.0
 19339        1.0
 58819        0.0
 
 [107488 rows x 1 columns],
         oh_label
 110400       0.0
 16476        0.0
 149164       0.0
 110688       1.0
 63037        0.0
 ...          ...
 13675        1.0
 70077        0.0
 8214         1.0
 77502        0.0
 34621        0.0
 
 [26873 rows x 1 columns])

In [22]:
mega_set.dtypes

oh_label    float64
cleaned      object
dtype: object

In [23]:
vect = TfidfVectorizer(ngram_range=(1,3),min_df=15,max_features=500,stop_words='english')
X_train_fr = vect.fit_transform(X_train)
X_test_fr = vect.transform(X_test)

In [ ]:
classifier = BinaryRelevance(classifier=SVC())
print("initialized")
classifier.fit(X_train_fr,y_train)
print("Trained SVC")
predictions = classifier.predict(X_test_fr)
# clf = BinaryRelevance(classifier=RandomForestClassifier())
# print("random forest otw")
# clf.fit(X_train,y_train)
# print("Trained random forest")
# predictions_rf = clf.predict(X_test)
# print("XGB time")
# model = BinaryRelevance(classifier=xgboost.XGBClassifier())
# model.fit(X_train,y_train)
# print("trained XGB")
# predictions_xgb = model.predict(X_test)

initialized


In [ ]:
print("SVC")
score = roc_auc_score(predictions.toarray(),y_test)
print(score)
accuracy = accuracy_score(y_test, predictions.toarray())
print(accuracy)
# print('---------------------RF-----------------')
# score = roc_auc_score(predictions_rf.toarray(),y_test)
# print(score)
# accuracy = accuracy_score(y_test, predictions_rf.toarray())
# print(accuracy)
# print("--------------------XG--------------------")
# score = roc_auc_score(predictions_xgb.toarray(),y_test)
# print(score)
# accuracy = accuracy_score(y_test, predictions_xgb.toarray())
# print(accuracy)

In [27]:
with open('vect.pkl', 'wb') as f:
    pickle.dump(vect, f)
with open('vect.pkl', 'rb') as f:
    vectorizer = pickle.load(f)
vectorizer

TfidfVectorizer(max_features=500, min_df=15, ngram_range=(1, 3),
                stop_words='english')

In [28]:
with open('sexual_assault.pkl', 'wb') as f:
    pickle.dump(classifier, f)
with open('sexual_assault.pkl', 'rb') as f:
    clf_loaded = pickle.load(f)
clf_loaded

BinaryRelevance(classifier=SVC(), require_dense=[True, True])